In [1]:
import requests
import base64
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
# Credenciales de tu app
CLIENT_ID = 'caf47a6239b14d3ab7f4423557c6671d'
CLIENT_SECRET = '3181316f04d54f85a52aec7bad53e2e4'

# URL para obtener el token
url = 'https://accounts.spotify.com/api/token'

# Codifica tus credenciales
credentials = f"{CLIENT_ID}:{ CLIENT_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# Encabezados y datos
headers = {
  'Authorization': f'Basic {encoded_credentials}'}
data = {
  'grant_type': 'client_credentials'}

# Solicitar el token
response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
  token = response.json().get('access_token')
  print(f"Access Token: {token}")
else:
  print(f"Error: {response.status_code}")
  print(response.json())

Access Token: BQAN2jfMCOm68DbobtWg9sHVkGlYZAFCvrRKloWNrft7aNoHv-lL4e8RSuq6LwARP7d8LXVjN0FeBJ4ky0K1hIcqYDLWnCN_W0aGH77HVxI-fEOOeVs


In [11]:
# Función para buscar y extraer datos de Spotify
def get_spotify_data(sp, genre, year_range):
    results = sp.search(q=f'genre:{genre} year:{year_range}', type='album,track')
    tracks = results['tracks']['items']
    albums = results['albums']['items']

    data = []
    for item in tracks + albums:
        if item['type'] == 'track':
            year = item['album']['release_date'][:4]
        else:
            year = item['release_date'][:4]

        data.append({
            'artist': item['artists'][0]['name'],
            'genre': genre,
            'type': item['type'],
            'name': item['name'],
            'year': year,
            'id': item['id']
        })
    return data

In [8]:
sp = spotipy.Spotify(auth='BQAN2jfMCOm68DbobtWg9sHVkGlYZAFCvrRKloWNrft7aNoHv-lL4e8RSuq6LwARP7d8LXVjN0FeBJ4ky0K1hIcqYDLWnCN_W0aGH77HVxI-fEOOeVs')

In [9]:
# Parámetros de búsqueda
start_year = 2020
end_year = 2024
genres = ['rock', 'pop', 'flamenco', 'electronic']

In [12]:
# Obtener los datos
all_data = []
for genre in genres:
    for year in range(start_year, end_year + 1):
        year_range = f"{year}-{year}"
        all_data.extend(get_spotify_data(sp, genre, year_range))

In [16]:
# Crear un DataFrame de Pandas.
df = pd.DataFrame(all_data)
print(df)

                          artist       genre   type  \
0                    Arde Bogotá        rock  track   
1                    Arde Bogotá        rock  track   
2                   David Bisbal        rock  track   
3                   David Bisbal        rock  track   
4                 Jarabe De Palo        rock  track   
..                           ...         ...    ...   
217  Happy Birthday to You Music  electronic  album   
218                Genre Juggler  electronic  album   
219                Genre-Benders  electronic  album   
220                Genre-Benders  electronic  album   
221                    Nivanoise  electronic  album   

                                                  name  year  \
0                                            Antiaéreo  2020   
1                                     Virtud y Castigo  2020   
2                                      A Partir De Hoy  2020   
3                                     Si Tú La Quieres  2020   
4                  

In [ ]:
# Eliminar duplicados basados en una columna específica.
df_sin_duplicados = df.drop_duplicates(subset=['artist'])

print(df_sin_duplicados)

                          artist       genre   type  \
0                    Arde Bogotá        rock  track   
2                   David Bisbal        rock  track   
4                 Jarabe De Palo        rock  track   
5                  Glass Animals        rock  track   
6           Guitarricadelafuente        rock  track   
..                           ...         ...    ...   
213                      Justice  electronic  track   
214                 Space Motion  electronic  track   
217  Happy Birthday to You Music  electronic  album   
218                Genre Juggler  electronic  album   
221                    Nivanoise  electronic  album   

                                                  name  year  \
0                                            Antiaéreo  2020   
2                                      A Partir De Hoy  2020   
4                                    Eso Que Tú Me Das  2020   
5                                           Heat Waves  2020   
6                  

In [17]:
# Guardar el DataFrame en un archivo CSV.
df_sin_duplicados.to_csv('datos_spotify.csv', index=False)

In [21]:
df_sin_duplicados_spotify = pd.read_csv("datos_spotify.csv")

In [22]:
df_sin_duplicados_spotify. head(200)

,artist,genre,type,name,year,id
0,Arde Bogotá,rock,track,Antiaéreo,2020,15tDDogDQ8kOcs5b6G2bN5
1,David Bisbal,rock,track,A Partir De Hoy,2020,3Z7JMhKrVBiMvMd0xV1vyI
2,Jarabe De Palo,rock,track,Eso Que Tú Me Das,2020,40tFJtuES1da2fg0OurUdI
3,Glass Animals,rock,track,Heat Waves,2020,3USxtqRwSYz57Ewm6wWRMp
4,Guitarricadelafuente,rock,track,Conticinio,2020,0GqylPxaT9i7pg6Kk0w8tm
...,...,...,...,...,...,...
107,Justice,electronic,track,Neverender,2024,3aQ9MHkMeL7Yu7jpyF62xn
108,Space Motion,electronic,track,Baiana - Radio Edit,2024,3ZbWQHFEZGl77HC2a3berD
109,Happy Birthday to You Music,electronic,album,House Music: Exploring the Dynamic World of El...,2024,2vTPhXMSnrADlsIkhBR5uC
110,Genre Juggler,electronic,album,Electronic Upbeat,2024,2vYD4qW7tEXa1mQQX0nJtp
